# Capstone Project Data Clean and Process Workbook
Cary Mosley, May 2020

## Import Libraries

In [1]:
import pandas as pd
import calendar, time
import datetime as dt
from datetime import date


## Data Clean

### Clean NYT Article Data

In [52]:
#Set Years
years = range(2005,2021)

#Select keywords for articles
keywords = ['Stock','Market','Finance','Business','Price','Debt','Portfolio','SP500',
            'Nasdaq','Dow Jones']

#Create DataFrame
nyt_df_final = pd.DataFrame(columns=['Date', 'Headline','Snippet'])

#Select just the headlines and snippets containing keywords
for year in years:
    
    try:
        #read in csv data
        path = 'nyt'+str(year)+'1.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'1')
    except:
        print('error',year,'1')
        continue
    
    try:
        #read in csv data
        path = 'nyt'+str(year)+'2.csv'
        nyt_df = pd.read_csv(path,index_col=None)
        nyt_df.drop(columns=['Unnamed: 0'],inplace=True)
        
        #select date and convert to datetime object
        nyt_df['Date']=nyt_df['Date'].apply(lambda x: x[0:10])
        nyt_df['Date']=pd.to_datetime(nyt_df['Date'])
        
        #all headlines are there but some snippets are missing
        nyt_df.dropna(inplace=True)
        
        #select headlines and snippets containing the keywords
        nyt_df_headlines = nyt_df[nyt_df['Headline'].str.contains('|'.join(keywords))]
        nyt_df_headlines = nyt_df_headlines[~nyt_df_headlines["Headline"].str.contains('Corrections')]
        nyt_df_snippets = nyt_df[nyt_df['Snippet'].str.contains('|'.join(keywords))]
        
        #concatenate dataframes
        nyt_df_final = pd.concat([nyt_df_final,nyt_df_headlines,nyt_df_snippets])
        print(year,'2')
    except:
        print('error',year,'2')
        continue

#Set the nyt_df_final index as the Date
nyt_df_final.set_index('Date', inplace=True)

#Write to pickle
nyt_df_final.to_pickle('nyt_df')

2005 1
error 2005 2
2006 1
error 2006 2
2007 1
2007 2
2008 1
2008 2
error 2009 1
error 2010 1
error 2011 1
error 2012 1
error 2013 1
error 2014 1
error 2015 1
error 2016 1
error 2017 1
error 2018 1
error 2019 1
error 2020 1


### Clean Ticker Data

In [55]:
#Import Ticker Data
spy_df = pd.read_csv('spy.csv')
vix_df = pd.read_csv('vix.csv')
vxx_df = pd.read_csv('vxx.csv')
qqq_df = pd.read_csv('qqq.csv')
iwm_df = pd.read_csv('iwm.csv')

In [56]:
spy_df.head()

,Date,Adj Close,Volume
0,2020-04-29,293.209991,118745600
1,2020-04-28,285.730011,105270000
2,2020-04-27,287.049988,77896600
3,2020-04-24,282.970001,85166000
4,2020-04-23,279.079987,104709700


In [57]:
spy_df.set_index('Date', inplace=True)

In [60]:
spy_df.isna().sum()

Adj Close    0
Volume       0
dtype: int64